In [1]:
import numpy as np
from glob import glob
from tqdm import tqdm
import os

In [2]:
sub_id = 18

In [3]:
### !!! IMPORTANT: The last 9 frames of all videos were cut before.

### # Concatenate All RFsignals

In [4]:
set_t = 'training'
sub_id = '18'
roi = 'VIS'

signals = []
for run_i in range(1,9):
    signals_run_i = np.transpose(np.load(f'{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
    signals.append(signals_run_i)
    
all_training_RFsignals = np.concatenate(signals).astype('float32')

np.save(f'{set_t}_signalsRF/sub-{sub_id}/{roi}/all_training_RFsignals', all_training_RFsignals)
all_training_RFsignals.shape

(3599, 216)

In [5]:
set_t = 'testing'
sub_id = '18'
roi = 'VIS'

signals = []
for run_i in range(1,9):
    signals_run_i = np.transpose(np.load(f'{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
    signals.append(signals_run_i)
    
all_testing_RFsignals = np.concatenate(signals).astype('float32')

np.save(f'{set_t}_signalsRF/sub-{sub_id}/{roi}/all_testing_RFsignals', all_testing_RFsignals)
all_testing_RFsignals.shape

(3599, 216)

## = Case 1: 
Just Taking the Last Run as Test Set (ignoring similar* and overlapping frames**)

In [6]:
numberOfFrames = [451, 441, 438, 488, 462, 439, 543, 337] 
numberOfFrames = np.array(numberOfFrames) - 9
numberOfFrames

# Except the last 2 runs, the final target frame numbers correspond to the number of fMRI volumes. 
# However, the targets and RFsignals have the equal number of row for each run at the end.
# probably because of downsizing the videos accordingly 2s TR

array([442, 432, 429, 479, 453, 430, 534, 328])

In [7]:
idxOfFrames = []
record = 0

for n in numberOfFrames:
    idxOfFrames.append(n+record)
    record += n 

idxOfFrames

[442, 874, 1303, 1782, 2235, 2665, 3199, 3527]

### # Targets

In [8]:
training_targets = np.load(f'training_videos/train_targets_warped.npy')
training_targets.shape

(3527, 96, 96, 3)

In [9]:
final_training_targets = training_targets[:idxOfFrames[6],:,:,:]

os.makedirs(f'Trials/sub-{sub_id}', exist_ok=True)
os.makedirs(f'Trials/sub-{sub_id}/Case1', exist_ok=True)
np.save(f'Trials/sub-{sub_id}/Case1/train_targets_case1.npy', np.array(final_training_targets))

final_training_targets.shape

(3199, 96, 96, 3)

In [10]:
testing_targets = np.load(f'testing_videos/test_targets_warped.npy')
testing_targets.shape

(3527, 96, 96, 3)

In [11]:
final_testing_targets = testing_targets[idxOfFrames[6]:idxOfFrames[7],:,:,:]

os.makedirs(f'Trials/sub-{sub_id}', exist_ok=True)
os.makedirs(f'Trials/sub-{sub_id}/Case1', exist_ok=True)
np.save(f'Trials/sub-{sub_id}/Case1/test_targets_case1.npy', np.array(final_testing_targets))

final_testing_targets.shape

(328, 96, 96, 3)

### # RFsignals

In [12]:
def final_RFsignals(sub_id, set_t, roi, test_run, time_range=5, shift=4):
    
    '''
    
    time_range: how many frames you want as input in the time dimension.
    shift: how many delays in frames.
    test_run: select the run number for the test set
    
    '''
    rel_dir = '/home/soyorh/Desktop/studyforrest/Preprocessing steps/'
    
    if set_t == 'training':
        
        number_of_run_files = len(glob(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run*'))
        signals_list = []

        for run_i in tqdm(range(1, number_of_run_files+1)):
            
            if run_i == test_run:
                continue
                
            # leftOut_frames = shift + time_range  ('Cause fmri duration == movie duration) already done in target space
            seen_img =  np.load(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run{run_i}.npy') #[:-leftOut_frames]
            n_frames = seen_img.shape[0]
            signal_selection = np.arange(time_range)[np.newaxis, :] + np.arange(shift, shift+n_frames)[:, np.newaxis]

            # get_signals_from_run(sub_id, set_t, roi, run_i, signal_selection)
            signals_run_i = np.transpose(np.load(f'{rel_dir}{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
            signals_run_i = signals_run_i[signal_selection,  :]

            signals_list.append(signals_run_i)

            print(f'training_signals_{run_i}: {signals_run_i.shape}')

        signals = np.concatenate(signals_list).astype('float32')
        signals = signals[:, :, :, np.newaxis, np.newaxis].astype(np.float32)
 
        print(f'training_signals.shape: {signals.shape}')
    
    elif set_t == 'testing':
        
        run_i = test_run
                    
        # leftOut_frames = shift + time_range  ('Cause fmri duration == movie duration) already done in target space
        seen_img =  np.load(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run{run_i}.npy') #[:-leftOut_frames]
        n_frames = seen_img.shape[0]
        signal_selection = np.arange(time_range)[np.newaxis, :] + np.arange(shift, shift+n_frames)[:, np.newaxis]

        # get_signals_from_run(sub_id, set_t, roi, run_i, signal_selection)
        signals_run_i = np.transpose(np.load(f'{rel_dir}{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
        signals_run_i = signals_run_i[signal_selection,  :]

        print(f'testing_signals_{run_i}: {signals_run_i.shape}')

        signals = signals_run_i.astype('float32')
        signals = signals[:, :, :, np.newaxis, np.newaxis].astype(np.float32)
 
        print(f'testing_signals.shape: {signals.shape}')

    return signals

In [13]:
set_t = 'training'
sub_id = '18'
roi = 'VIS'
test_run = 8

final_training_RFsignals = final_RFsignals(sub_id, set_t, roi, test_run, time_range=5, shift=4)

np.save(f'Trials/sub-{sub_id}/Case1/train_RFsignals_case1.npy', np.array(final_training_RFsignals))

100%|██████████| 8/8 [00:00<00:00, 159.15it/s]

training_signals_1: (442, 5, 216)
training_signals_2: (432, 5, 216)
training_signals_3: (429, 5, 216)
training_signals_4: (479, 5, 216)
training_signals_5: (453, 5, 216)
training_signals_6: (430, 5, 216)
training_signals_7: (534, 5, 216)
training_signals.shape: (3199, 5, 216, 1, 1)


In [14]:
set_t = 'testing'
sub_id = '18'
roi = 'VIS'
test_run = 8

final_testing_RFsignals = final_RFsignals(sub_id, set_t, roi, test_run, time_range=5, shift=4)

np.save(f'Trials/sub-{sub_id}/Case1/test_RFsignals_case1.npy', np.array(final_testing_RFsignals))

testing_signals_8: (328, 5, 216)
testing_signals.shape: (328, 5, 216, 1, 1)


## = Case 2: 
Dividing the test into the chunks of 160 (# In Total 22 chunks. The last one has 167 frames).
Finding the chunk that has the minumum number of SSIM>=0.5.
In our case it is the 6th chunk.

6th chunk is the test set. The others are the training set.

In [15]:
# In Total 22 chunks. The last one has 167 frames.
Total = 21*160 + 167
Total

3527

In [16]:
chunks = []
for i in range(22):
    chunks.append(160)
    
chunks[21] = 167

In [17]:
len(chunks)

22

In [18]:
idxOfChunks = []
record = 0

for n in chunks:
    idxOfChunks.append(n+record)
    record += n 

idxOfChunks

[160,
 320,
 480,
 640,
 800,
 960,
 1120,
 1280,
 1440,
 1600,
 1760,
 1920,
 2080,
 2240,
 2400,
 2560,
 2720,
 2880,
 3040,
 3200,
 3360,
 3527]

### # Targets

In [19]:
training_targets = np.load(f'training_videos/train_targets_warped.npy')
training_targets.shape

(3527, 96, 96, 3)

In [20]:
n = 6 # choose the test chunk to be excluded
i_chunk = n - 1 

before_chunk = training_targets[:idxOfChunks[i_chunk-1],:,:,:]
after_chunk = training_targets[idxOfChunks[i_chunk]:Total,:,:,:]

final_training_targets = np.concatenate([before_chunk, after_chunk])

os.makedirs(f'Trials/sub-{sub_id}/Case2', exist_ok=True)
np.save(f'Trials/sub-{sub_id}/Case2/train_targets_case2.npy', np.array(final_training_targets))

final_training_targets.shape

(3367, 96, 96, 3)

In [21]:
testing_targets = np.load(f'testing_videos/test_targets_warped.npy')
testing_targets.shape

(3527, 96, 96, 3)

In [22]:
n = 6 # choose the test chunk
i_chunk = n - 1 

final_testing_targets = testing_targets[idxOfChunks[i_chunk-1]:idxOfChunks[i_chunk],:,:,:]

os.makedirs(f'Trials/sub-{sub_id}/Case2', exist_ok=True)
np.save(f'Trials/sub-{sub_id}/Case2/test_targets_case2.npy', np.array(final_testing_targets))

final_testing_targets.shape

(160, 96, 96, 3)

### # RFsignals

#### -> all_RFsignals

In [23]:
def helper_RFsignals_case2(sub_id, set_t, roi, time_range=5, shift=4):
    
    '''
    
    time_range: how many frames you want as input in the time dimension.
    shift: how many delays in frames.
    test_run: select the run number for the test set
    
    '''
    rel_dir = '/home/soyorh/Desktop/studyforrest/Preprocessing steps/'
    
    number_of_run_files = len(glob(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run*'))
    signals_list = []

    for run_i in tqdm(range(1, number_of_run_files+1)):

        # leftOut_frames = shift + time_range  ('Cause fmri duration == movie duration) already done in target space
        seen_img =  np.load(f'{rel_dir}{set_t}_videos/{set_t[:-3]}_video_run{run_i}.npy') #[:-leftOut_frames]
        n_frames = seen_img.shape[0]
        signal_selection = np.arange(time_range)[np.newaxis, :] + np.arange(shift, shift+n_frames)[:, np.newaxis]

        # get_signals_from_run(sub_id, set_t, roi, run_i, signal_selection)
        signals_run_i = np.transpose(np.load(f'{rel_dir}{set_t}_signalsRF/sub-{sub_id}/{roi}/{run_i}_{set_t[:-3]}.npy'))
        signals_run_i = signals_run_i[signal_selection,  :]

        signals_list.append(signals_run_i)

        print(f'training_signals_run_i: {signals_run_i.shape}')

    signals = np.concatenate(signals_list).astype('float32')
    signals = signals[:, :, :, np.newaxis, np.newaxis].astype(np.float32)

    print(f'training_signals.shape: {signals.shape}')

    return signals

In [24]:
set_t = 'training' # or 'testing': they are the same.
sub_id = '18'
roi = 'VIS'

all_RFsignals = helper_RFsignals_case2(sub_id, set_t, roi, time_range=5, shift=4)

100%|██████████| 8/8 [00:00<00:00, 140.84it/s]

training_signals_run_i: (442, 5, 216)
training_signals_run_i: (432, 5, 216)
training_signals_run_i: (429, 5, 216)
training_signals_run_i: (479, 5, 216)
training_signals_run_i: (453, 5, 216)
training_signals_run_i: (430, 5, 216)
training_signals_run_i: (534, 5, 216)
training_signals_run_i: (328, 5, 216)
training_signals.shape: (3527, 5, 216, 1, 1)


In [25]:
all_RFsignals.shape

(3527, 5, 216, 1, 1)

#### -> chunks_RFsignals

In [26]:
def final_RFsignals_chunks(sub_id, set_t, roi, test_chunk, chunks_RFsignals):
    
    '''
    
    time_range: how many frames you want as input in the time dimension.
    shift: how many delays in frames.
    test_chunk: select the chunk number for the test set
    
    '''   
    
    test_chunk -= 1
    
    if set_t == 'training':
        
        signals_list = []

        for chunk_i in tqdm(range(len(chunks_RFsignals))):
            
            if chunk_i == test_chunk:
                continue

            chunk_RFsignal = chunks_RFsignals[chunk_i]
            signals_list.append(chunk_RFsignal)

            print(f'training_signals_{chunk_i+1}: {chunk_RFsignal.shape}')

        signals = np.concatenate(signals_list)
 
        print(f'training_signals.shape: {signals.shape}')
    
    elif set_t == 'testing':
        
        chunk_i = test_chunk
        signals = chunks_RFsignals[chunk_i]
 
        print(f'testing_signals_{chunk_i+1}: {signals.shape}')

    return signals

In [27]:
chunks_RFsignals = []
N = 0
    
for i_chunk in range(len(idxOfChunks)):

    chunk_RFsignal = all_RFsignals[N:idxOfChunks[i_chunk],:,:,:]
    chunks_RFsignals.append(chunk_RFsignal)
    
    N = idxOfChunks[i_chunk]

In [28]:
len(chunks_RFsignals), chunks_RFsignals[0].shape, chunks_RFsignals[21].shape

(22, (160, 5, 216, 1, 1), (167, 5, 216, 1, 1))

In [29]:
set_t = 'training'
sub_id = '18'
roi = 'VIS'
test_chunk = 6

final_training_RFsignals = final_RFsignals_chunks(sub_id, set_t, roi, test_chunk, chunks_RFsignals)

np.save(f'Trials/sub-{sub_id}/Case2/train_RFsignals_case2.npy', np.array(final_training_RFsignals))

100%|██████████| 22/22 [00:00<00:00, 25476.17it/s]


training_signals_1: (160, 5, 216, 1, 1)
training_signals_2: (160, 5, 216, 1, 1)
training_signals_3: (160, 5, 216, 1, 1)
training_signals_4: (160, 5, 216, 1, 1)
training_signals_5: (160, 5, 216, 1, 1)
training_signals_7: (160, 5, 216, 1, 1)
training_signals_8: (160, 5, 216, 1, 1)
training_signals_9: (160, 5, 216, 1, 1)
training_signals_10: (160, 5, 216, 1, 1)
training_signals_11: (160, 5, 216, 1, 1)
training_signals_12: (160, 5, 216, 1, 1)
training_signals_13: (160, 5, 216, 1, 1)
training_signals_14: (160, 5, 216, 1, 1)
training_signals_15: (160, 5, 216, 1, 1)
training_signals_16: (160, 5, 216, 1, 1)
training_signals_17: (160, 5, 216, 1, 1)
training_signals_18: (160, 5, 216, 1, 1)
training_signals_19: (160, 5, 216, 1, 1)
training_signals_20: (160, 5, 216, 1, 1)
training_signals_21: (160, 5, 216, 1, 1)
training_signals_22: (167, 5, 216, 1, 1)
training_signals.shape: (3367, 5, 216, 1, 1)


In [30]:
set_t = 'testing'
sub_id = '18'
roi = 'VIS'
test_chunk = 6

final_testing_RFsignals = final_RFsignals_chunks(sub_id, set_t, roi, test_chunk, chunks_RFsignals)

np.save(f'Trials/sub-{sub_id}/Case2/test_RFsignals_case2.npy', np.array(final_testing_RFsignals))

testing_signals_6: (160, 5, 216, 1, 1)


## = Case ?: 
type here the explanation 